#### Level-4 data of L1000 gene-expression profiling is calculated as:

Level 4 - Differential Expression (ZSPC):

To obtain a measure of relative gene expression, we use a robust z-scoring procedure to generate differential expression values from normalized profiles (Level-3). We compute the **differential expression of gene x in the ith sample(xi) on the plate (zi)** as
                              
                              zi = xi - median(X) / 1.4826 * MAD(X)


***where X is the vector of normalized gene expression of gene x across all samples on the plate, MAD is the median absolute deviation of X, and the factor of 1.4826 makes the denominator a consistent estimator of scale for normally distributed data.***

[Subramanian et al 2017](https://www.cell.com/action/showPdf?pii=S0092-8674%2817%2931309-0)



- Level-4 data - are replicate level data i.e. where you have multiple profiles been perturbed by the same compound (pertubagen)

#### The goal here:


-- is to determine the median score of each compound per dose based on taking the median of the correlation values between replicates of the same compound.

In [1]:
import os
import pathlib
import requests
import pickle
import argparse
import pandas as pd
import numpy as np
import re
from os import walk
from collections import Counter
import random
import shutil
from statistics import median
import cmapPy.pandasGEXpress.parse_gct as pg
from cmapPy.pandasGEXpress.parse import parse
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

In [2]:
data_dir = pathlib.Path("../L1000/L1000_figshare_data")
os.listdir(data_dir) ##files in L1000 downloaded dataset

['level_4_zspc_n27837x978.gctx',
 'level_4W_zspc_n27837x978.gctx',
 'level_3_q2norm_n27837x978.gctx',
 'level_5_rank_n9482x978.gctx',
 'level_5_modz_n9482x978.gctx',
 'col_meta_level_5_n169494.txt',
 'level_5_modz_cid_n9482.grp',
 'set_size_3_level_3_REP.A_A549_only_all_compounds_and_random_well_DMSO.gmt',
 'col_meta_level_3_REP.A_A549_only_n27837.txt',
 'REP.A_A549_pert_info.txt',
 'level_5_modz_common_sigs_n8370x978.gctx',
 'col_meta_level_3_n421176.txt',
 'col_meta_level_5_REP.A_A549_only_n9482.txt']

In [3]:
pertinfo_file = '../aligned_moa_CP_L1000.csv'

Two level-4 L1000 Data:

- 'level_4W_zspc_n27837x978.gctx'
- 'level_4_zspc_n27837x978.gctx',

In [4]:
# Run either using "W" or ""
# Representing "whitened" (aka "spherized") L1000 data or not
l1000_file_indicator = ""

In [5]:
def construct_lvl4_data(data_dir, level4_dir, pertinfo_file):
    """
    This function returns L1000 Level-4 Data that is aligned with 
    the important metadata information to compute compound's median scores
    """
    
    lvl4_data = parse(os.path.join(data_dir, level4_dir))
    lvl4_data = lvl4_data.data_df.rename_axis(None).T
    lvl4_data = lvl4_data.rename_axis(None).reset_index()
    lvl4_data.rename(columns={"index": "replicate_id"}, inplace = True)
    df_metalvl_5 = pd.read_csv(os.path.join(data_dir, 'col_meta_level_5_REP.A_A549_only_n9482.txt'), delimiter = "\t")
    lvl4_data['sig_id'] = lvl4_data['replicate_id'].apply(lambda x: ''.join(re.sub('(24H.+(\_|\.)[A-Z0-9]+.)\:', '24H:', x)))
    df_meta_features = df_metalvl_5[['sig_id', 'pert_id', 'pert_idose', 'det_plate', 'det_well']].copy()
    df_meta_features['dose'] = df_meta_features['pert_idose'].map({'-666' : 0, '0.04 uM' : 1, '0.12 uM' : 2, '0.37 uM' : 3,
                                                                   '1.11 uM' : 4, '3.33 uM' : 5, '10 uM' : 6, '20 uM' : 7})
    df_pertinfo = pd.read_csv(pertinfo_file)
    df_pertinfo.rename(columns={"broad_id": "pert_id"}, inplace = True)
    df_meta_features = pd.merge(df_meta_features, df_pertinfo, on=['pert_id'], how = 'left')
    lvl4_data = pd.merge(lvl4_data, df_meta_features, on='sig_id')
    
    return lvl4_data

In [6]:
# Load compound median scores
if l1000_file_indicator == "W":
    file = 'level_4W_zspc_n27837x978.gctx'
else:
    file = 'level_4_zspc_n27837x978.gctx'

df_level4 = construct_lvl4_data(data_dir, file, pertinfo_file)

print(df_level4.shape)
df_level4.head()

(27837, 988)


,replicate_id,200814_at,222103_at,201453_x_at,204131_s_at,200059_s_at,205067_at,213702_x_at,214435_x_at,201334_s_at,...,205379_at,sig_id,pert_id,pert_idose,det_plate,det_well,dose,Metadata_broad_sample,pert_iname,moa
0,REP.A001_A549_24H_X1_B27:A03,0.3547,-0.4940,-0.1721,-0.0339,-0.4355,1.8263,-0.1316,0.0853,-0.4660,...,0.1046,REP.A001_A549_24H:A03,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A03,0,DMSO,DMSO,Control vehicle
1,REP.A001_A549_24H_X2_B27:A03,-0.0447,-1.6390,-0.5276,-0.5092,-0.5733,0.2445,0.6159,-1.1273,2.8250,...,-1.5836,REP.A001_A549_24H:A03,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A03,0,DMSO,DMSO,Control vehicle
2,REP.A001_A549_24H_X3_B27:A03,-0.9583,0.2657,-0.8204,-0.5851,-2.0808,0.0739,-0.0497,-0.3012,-0.9787,...,-0.1416,REP.A001_A549_24H:A03,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A03,0,DMSO,DMSO,Control vehicle
3,REP.A001_A549_24H_X1_B27:A04,-0.2130,0.4931,-0.8768,-0.6968,-1.7018,-0.3779,-0.6745,-1.9799,-1.1429,...,-1.8243,REP.A001_A549_24H:A04,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A04,0,DMSO,DMSO,Control vehicle
4,REP.A001_A549_24H_X2_B27:A04,0.7499,-1.2819,0.4981,1.7090,1.6765,-1.2690,1.7974,-1.2213,3.4625,...,0.3157,REP.A001_A549_24H:A04,DMSO,-666,REP.A001_A549_24H_X1_B27|REP.A001_A549_24H_X2_...,A04,0,DMSO,DMSO,Control vehicle


In [7]:
len(df_level4['det_plate'].unique())

128

In [8]:
len(df_level4['det_well'].unique())

380

### - Remove highly correlated landmark genes and samples with Null compound values

In [9]:
def feature_selection(df_data):
    
    """
    Perform feature selection by dropping columns with null compounds values, 
    and highly correlated landmark genes from the data.
    """
    
    df_data_genes = df_data.drop(['replicate_id', 'Metadata_broad_sample', 'pert_id', 'dose', 'pert_idose', 
                                  'pert_iname', 'moa', 'sig_id', 'det_plate', 'det_well'], axis = 1).copy()
    df_data_corr = df_data_genes.corr(method = 'spearman')
    drop_cols = []
    n_cols = len(df_data_corr.columns)
    for i in range(n_cols):
        for k in range(i+1, n_cols):
            val = df_data_corr.iloc[k, i]
            col = df_data_corr.columns[i]
            if abs(val) >= 0.9:
                drop_cols.append(col)
    df_data.drop(set(drop_cols), axis = 1, inplace = True)
    df_data.drop(df_data[df_data['pert_iname'].isnull()].index).reset_index(drop = True, inplace = True)
    
    return df_data

In [10]:
df_level4 = feature_selection(df_level4)

In [11]:
df_level4.shape

(27837, 988)

In [12]:
def get_median_score(cpds_list, df):
    """
    This function calculates the median score for each compound based on its replicates
    """
    
    cpds_median_score = {}
    for cpd in cpds_list:
        cpd_replicates = df[df['pert_iname'] == cpd].copy()
        cpd_replicates.drop(['replicate_id', 'Metadata_broad_sample', 'pert_id', 'dose', 'pert_idose', 
                             'pert_iname', 'moa', 'sig_id', 'det_plate', 'det_well'], axis = 1, inplace = True)
        cpd_replicates_corr = cpd_replicates.astype('float64').T.corr(method = 'pearson').values
        if len(cpd_replicates_corr) == 1:
            median_val = 1
        else:
            median_val = median(list(cpd_replicates_corr[np.triu_indices(len(cpd_replicates_corr), k = 1)]))
        
        cpds_median_score[cpd] = median_val
        
    return cpds_median_score

In [13]:
def check_compounds(cpd_med_score, df):
    """
    Check if all distinct compounds in the Level-4 dataframe are present 
    in the cpd_med_score dictionary, if not add the compounds as keys to the dictionary 
    and give them a null value.
    """
    cpd_list = df['pert_iname'].unique().tolist()
    cpd_keys = cpd_med_score.keys()
    for cpd in cpd_list:
        if cpd not in cpd_keys:
            cpd_med_score[cpd] = np.nan
            
    return cpd_med_score

In [14]:
def get_cpd_medianscores(df):
    
    """This function computes median scores for all compounds found in the Level-4 dataframe PER DOSE (1-6)"""
    
    dose_list = list(set(df['dose'].unique().tolist()))[1:7]
    
    for dose in dose_list:
        df_dose = df[df['dose'] == dose].copy()
        cpds_list = df_dose['pert_iname'].unique().tolist()
        cpds_median_score = get_median_score(cpds_list, df_dose)
        cpds_median_score = check_compounds(cpds_median_score, df)
        sorted_med_score = {key:value for key, value in sorted(cpds_median_score.items(), key=lambda item: item[0])}
        if dose == 1:
            df_cpd_med_score = pd.DataFrame.from_dict(sorted_med_score, orient='index', columns = ['dose_1'])
        else:
            df_cpd_med_score['dose_' + str(dose)] = sorted_med_score.values()
            
    return df_cpd_med_score

In [15]:
df_cpd_med_score = get_cpd_medianscores(df_level4)

In [16]:
df_cpd_med_score.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6
"16,16-dimethylprostaglandin-e2",0.096687,0.043230,0.201810,0.210823,NaN,NaN
17-hydroxyprogesterone-caproate,0.073370,0.064999,0.022398,0.149151,0.031656,0.226888
2-iminobiotin,0.085434,0.196718,0.012828,0.005784,0.046477,0.128650
"3,3'-diindolylmethane",0.013985,-0.031721,0.024554,0.070705,NaN,NaN
3-amino-benzamide,0.011228,-0.000293,0.052259,0.013569,-0.013665,0.021126
3-deazaadenosine,0.068822,0.001870,0.045240,0.204147,0.061483,0.049968
ABT-737,0.025780,0.010465,-0.007462,-0.005342,0.006452,0.111000
AICA-ribonucleotide,0.003928,0.030120,0.017287,-0.004123,0.054308,0.057677
AKT-inhibitor-1-2,0.063741,0.099233,0.080534,0.078303,0.069667,0.054751
ALX-5407,0.017286,0.009509,0.034342,-0.020208,0.029120,0.046782


In [17]:
def drop_cpds_with_null(df):
    """
    This function drop compounds with median scores of 1 
    or null values in any of the dose points (1-6)
    """
    cpds_with_null = []
    for cpd in df.index:
        if any(df.loc[cpd] == 1) | any(df.loc[cpd].isnull()):
            cpds_with_null.append(cpd)
    df.drop(cpds_with_null, axis = 0, inplace = True)
    
    return df

In [18]:
df_cpd_med_score = drop_cpds_with_null(df_cpd_med_score)

In [19]:
df_cpd_med_score.shape

(1330, 6)

In [20]:
df_cpd_med_score.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6
17-hydroxyprogesterone-caproate,0.073370,0.064999,0.022398,0.149151,0.031656,0.226888
2-iminobiotin,0.085434,0.196718,0.012828,0.005784,0.046477,0.128650
3-amino-benzamide,0.011228,-0.000293,0.052259,0.013569,-0.013665,0.021126
3-deazaadenosine,0.068822,0.001870,0.045240,0.204147,0.061483,0.049968
ABT-737,0.025780,0.010465,-0.007462,-0.005342,0.006452,0.111000
AICA-ribonucleotide,0.003928,0.030120,0.017287,-0.004123,0.054308,0.057677
AKT-inhibitor-1-2,0.063741,0.099233,0.080534,0.078303,0.069667,0.054751
ALX-5407,0.017286,0.009509,0.034342,-0.020208,0.029120,0.046782
AS-605240,-0.088274,0.042611,-0.072933,-0.089080,0.052919,0.057996
AT-7519,0.090320,0.000638,0.039465,0.236131,0.799360,0.840282


In [21]:
def no_of_replicates_per_cpd(df, df_lvl4):
    """This function computes the numbers of replicates for each compound (cpd_size)"""
    
    dose_list = list(set(df_lvl4['dose'].unique().tolist()))[1:7]
    cpds_no_of_reps = {}
    for cpd in df.index:
        num_of_reps = 0
        df_cpd = df_lvl4[df_lvl4['pert_iname'] == cpd].copy()
        for dose in dose_list:
            df_dose = df_cpd[df_cpd['dose'] == dose].copy()
            num_of_reps += df_dose.shape[0]
        cpds_no_of_reps[cpd] = num_of_reps // len(dose_list)
    df['no_of_replicates'] = cpds_no_of_reps.values()
    return df

In [22]:
df_cpd_med_score = no_of_replicates_per_cpd(df_cpd_med_score, df_level4)
df_cpd_med_score.head(10)

,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6,no_of_replicates
17-hydroxyprogesterone-caproate,0.073370,0.064999,0.022398,0.149151,0.031656,0.226888,3
2-iminobiotin,0.085434,0.196718,0.012828,0.005784,0.046477,0.128650,2
3-amino-benzamide,0.011228,-0.000293,0.052259,0.013569,-0.013665,0.021126,3
3-deazaadenosine,0.068822,0.001870,0.045240,0.204147,0.061483,0.049968,2
ABT-737,0.025780,0.010465,-0.007462,-0.005342,0.006452,0.111000,3
AICA-ribonucleotide,0.003928,0.030120,0.017287,-0.004123,0.054308,0.057677,3
AKT-inhibitor-1-2,0.063741,0.099233,0.080534,0.078303,0.069667,0.054751,3
ALX-5407,0.017286,0.009509,0.034342,-0.020208,0.029120,0.046782,3
AS-605240,-0.088274,0.042611,-0.072933,-0.089080,0.052919,0.057996,2
AT-7519,0.090320,0.000638,0.039465,0.236131,0.799360,0.840282,3


In [23]:
def save_to_csv(df, path, file_name, compress=None):
    """saves dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index=False, compression=compress)

In [24]:
# Load compound median scores
if l1000_file_indicator == "W":
    file = 'cpd_replicate_median_scores_W.csv'
else:
    file = 'cpd_replicate_median_scores.csv'

save_to_csv(df_cpd_med_score.reset_index().rename({'index':'cpd'}, axis = 1), 'L1000_lvl4_cpd_replicate_datasets', file)

In [25]:
# Load compound median scores
if l1000_file_indicator == "W":
    file = 'L1000_level4W_cpd_replicates.csv.gz'
else:
    file = 'L1000_level4_cpd_replicates.csv.gz'


save_to_csv(df_level4, 'L1000_lvl4_cpd_replicate_datasets', file, compress="gzip")

In [26]:
# Output files for visualization
results_dir = pathlib.Path("../results")
if l1000_file_indicator == "W":
    file = 'median_score_per_compound_L1000W'
else:
    file = 'median_score_per_compound_L1000'
    
cpd_summary_file = pathlib.Path(f"{results_dir}/{file}.tsv.gz")

dose_recode_info = {
    'dose_1': '0.04 uM', 'dose_2':'0.12 uM', 'dose_3':'0.37 uM',
    'dose_4': '1.11 uM', 'dose_5':'3.33 uM', 'dose_6':'10 uM'
}

In [27]:
cpd_score_summary_df = (
    df_cpd_med_score
    .reset_index()
    .rename(columns={"index": "compound"})
    .melt(
        id_vars=["compound", "no_of_replicates"],
        value_vars=["dose_1", "dose_2", "dose_3", "dose_4", "dose_5", "dose_6"],
        var_name="dose",
        value_name="median_replicate_score"
    )
)

cpd_score_summary_df.dose = cpd_score_summary_df.dose.replace(dose_recode_info)

cpd_score_summary_df.to_csv(cpd_summary_file, sep="\t", index=False)
cpd_score_summary_df.head()

,compound,no_of_replicates,dose,median_replicate_score
0,17-hydroxyprogesterone-caproate,3,0.04 uM,0.073370
1,2-iminobiotin,2,0.04 uM,0.085434
2,3-amino-benzamide,3,0.04 uM,0.011228
3,3-deazaadenosine,2,0.04 uM,0.068822
4,ABT-737,3,0.04 uM,0.025780
